# Code to Adjust Offensive, Defensive and Net Ratings for Strength of Schedule

In [ ]:
from sklearn.linear_model import RidgeCV
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

# from sklearn.pipeline import make_pipeline
# model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

teams_dict, teams_list = get_teams(league="NBA")
box_DIR = "../data/box/"

# Get Ratings Function:
1. Reads Advanced Box Scores from csv file
2. Groups two entries for a single game (team1 and team2)
3. Splits columns for team1 and team2 for team1
4. Splits columns for team1 and team2 for team2
5. Combines the split data to get back two entries for single game with appropriate opposition information and stats
6. Returns the combined data

In [ ]:
def get_ratings(season=2023,ngames=0):
    cols = [
        "gameId",
        "teamName",
        "teamId",
        "offensiveRating",
        "defensiveRating",
        "netRating",
        "possessions",
    ]
    df = pd.read_parquet(box_DIR + f"NBA_Box_T_Adv_{season}.parquet", columns=cols)
    cols = ["gameId", "team", "tId", "ORtg", "DRtg", "NRtg", "poss"]
    df.columns = cols
    df1 = df.groupby("gameId")
    df1_1 = df1.nth(0)
    df1_2 = df1.nth(1)
    df1_1.columns = ["gameId"] + [s + "1" for s in df1_1.columns if s != "gameId"]
    df1_2.columns = ["gameId"] + [s + "2" for s in df1_2.columns if s != "gameId"]
    df1_3 = pd.merge(df1_1, df1_2, on="gameId")
    df1_4 = df1.nth(1)
    df1_5 = df1.nth(0)
    df1_4.columns = ["gameId"] + [s + "1" for s in df1_4.columns if s != "gameId"]
    df1_5.columns = ["gameId"] + [s + "2" for s in df1_5.columns if s != "gameId"]
    df1_6 = pd.merge(df1_4, df1_5, on="gameId")
    df2 = pd.concat([df1_3, df1_6]).sort_values(by="gameId").reset_index(drop=True)
    data1 = df2.copy()
    stats = leaguegamelog.LeagueGameLog(
        player_or_team_abbreviation="T",
        season=season,
        season_type_all_star="Regular Season",
    )
    df10 = stats.get_data_frames()[0]
    if ngames != 0:
        df10g = df10.groupby("TEAM_NAME")
        df10 = df10g.nth(np.arange(-ngames,0,1)).reset_index(drop=True)
    df10["HOME"] = ~df10["MATCHUP"].str.contains("@")
    df10["tId1"] = df10["TEAM_ID"]
    df10["gameId"] = df10["GAME_ID"]
    df11 = (
        df10[["gameId", "tId1", "HOME"]].sort_values(by="gameId").reset_index(drop=True)
    )
    df11[["gameId", "tId1"]] = df11[["gameId", "tId1"]].astype(int)
    data = pd.merge(data1, df11)
    return data

# Process Results Function
1. Calculates Offensive, Defensive and Net Ratings from data
2. Merges Regression results with data (combine adjusted and unadjusted ratings)
3. Round all results
4. Calculate SOS values
5. Return results after cleaning and sorting

In [ ]:
def process_results(data, results_adj):
    data["pts1"] = data["ORtg1"] * data["poss1"]
    data["pts2"] = data["DRtg1"] * data["poss1"]
    off_prior = data.groupby(["tId1"])[["poss1", "pts1"]].agg("sum").reset_index()
    def_prior = data.groupby(["tId1"])[["poss1", "pts2"]].agg("sum").reset_index()
    off_prior["OFF"] = off_prior["pts1"] / off_prior["poss1"]
    off_prior = off_prior[["tId1", "OFF"]]
    def_prior["DEF"] = def_prior["pts2"] / def_prior["poss1"]
    def_prior = def_prior[["tId1", "DEF"]]
    results_net = pd.merge(off_prior, def_prior, on=["tId1"])
    results_net["NET"] = results_net["OFF"] - results_net["DEF"]
    results_net.rename(columns={"tId1": "tId"}, inplace=True)
    results_net = results_net.astype(float).round(2)
    results_net["tId"] = results_net["tId"].astype(int)
    ortg_mean = data["pts1"].sum() / data["poss1"].sum()
    drtg_mean = data["pts2"].sum() / data["poss1"].sum()
    results_adj["tId"] = results_adj["tId"].astype(int)
    results_comb = pd.merge(results_net, results_adj, on=["tId"])
    results_comb["aOFF"] = results_comb["aOFF"]
    results_comb["aDEF"] = results_comb["aDEF"]
    results_comb["oSOS"] = results_comb["aOFF"] - results_comb["OFF"]
    results_comb["dSOS"] = results_comb["DEF"] - results_comb["aDEF"]
    results_comb["SOS"] = results_comb["oSOS"] + results_comb["dSOS"]
    results_comb.iloc[:, 1:] = results_comb.iloc[:, 1:].round(1)
    results = results_comb[
        ["Team", "OFF", "oSOS", "aOFF", "DEF", "dSOS", "aDEF", "NET", "SOS", "aNET"]
    ]
    # results = results_comb[["Team","OFF","DEF","NET","aOFF","aDEF","aNET"]]
    results = results.sort_values(by="aNET", ascending=0).reset_index(drop=True)
    return results, ortg_mean, drtg_mean

# maps_teams function
1. Makes the matrix rows to be used in ridge regression
2. The weights for each team = 1/2
3. Equations per game are:  
0.5*team1off_est + 0.5*team2def_est = team1offrating  
0.5*team2off_est + 0.5*team1def_est = team2offrating  
4. The reason for doing this is that for unadjusted values of a game, team1offrating = team2defrating.  
So team1offrating = 0.5*team1offrating + 0.5*team2defrating. Therefore I use a similar structure for estimating adjusted ratings
# convert_to_matrices
1. Converts each row of data dataframe to x stints.
2. Then maps those rows using `map_teams` function to get matrix X rows
3. Gets Y rows
# lambda_to_alpha
Converts lambda value to alpha needed for ridge CV
# calculate_netrtg
1. Converts lambdas to alphas using `lambda_to_alpha` function
2. Defines the ridge regression problem using `scikit-learn`'s `RidgeCV` algorithm
3. `cv=5` is chosen i.e. kfold cross-validation splitting strategy using `k=5`
4. `Intercept` is set as true. This value is to be added later to our estimation results to get Offensive and Defensive ratings.
5. Gets coefficients and intercept
6. Add intercept to intercept to get adjusted ratings. Use adjusted off and def ratings to calculate adjusted net rating.
7. Create and return adjusted ratings dataframe

In [ ]:
def map_teams(row_in, teams, scale):
    t1 = row_in[0]
    t2 = row_in[1]

    rowOut = np.zeros([len(teams) * 2])
    rowOut[teams.index(t1)] = scale
    rowOut[teams.index(t2) + len(teams)] = scale

    return rowOut


def convert_to_matricies(possessions, name, teams, scale=1):
    # extract only the columns we need
    # Convert the columns of player ids into a numpy matrix
    stints_x_base = possessions[["tId1", "tId2"]].to_numpy()
    # Apply our mapping function to the numpy matrix
    stint_X_rows = np.apply_along_axis(map_teams, 1, stints_x_base, teams, scale=scale)
    # Convert the column of target values into a numpy matrix
    stint_Y_rows = possessions[name].to_numpy()

    # return matricies and possessions series
    return stint_X_rows, stint_Y_rows


# Convert lambda value to alpha needed for ridge CV


def lambda_to_alpha(lambda_value, samples):
    return (lambda_value * samples) / 2.0


# Convert RidgeCV alpha back into a lambda value


def alpha_to_lambda(alpha_value, samples):
    return (alpha_value * 2.0) / samples


def calculate_netrtg(train_x, train_y, lambdas, teams_list):
    alphas = [lambda_to_alpha(l, train_x.shape[0]) for l in lambdas]
    # create a 5 fold CV ridgeCV model. Our target data is not centered at 0, so we want to fit to an intercept.
    clf = RidgeCV(alphas=alphas, cv=5, fit_intercept=True)

    # fit our training data
    model = clf.fit(
        train_x,
        train_y,
    )

    # convert our list of players into a mx1 matrix
    team_arr = np.transpose(np.array(teams_list).reshape(1, len(teams_list)))

    # extract our coefficients into the offensive and defensive parts
    coef_offensive_array = model.coef_[0 : len(teams_list)][np.newaxis].T
    coef_defensive_array = model.coef_[len(teams_list) : 2 * len(teams_list)][
        np.newaxis
    ].T
    # concatenate the offensive and defensive values with the playey ids into a mx3 matrix
    team_id_with_coef = np.concatenate(
        [team_arr, coef_offensive_array, coef_defensive_array], axis=1
    )
    # build a dataframe from our matrix
    teams_coef = pd.DataFrame(team_id_with_coef)
    intercept = model.intercept_
    teams_coef.columns = ["tId", "aOFF", "aDEF"]
    teams_coef["aNET"] = teams_coef["aOFF"] - teams_coef["aDEF"]
    teams_coef["aOFF"] = teams_coef["aOFF"] + intercept
    teams_coef["aDEF"] = teams_coef["aDEF"] + intercept
    teams_coef["Team"] = teams_coef["tId"].map(teams_dict)
    results = teams_coef[["tId", "Team", "aOFF", "aDEF", "aNET"]]
    results = results.sort_values(by=["aNET"], ascending=False).reset_index(drop=True)
    return results, model, intercept

In [ ]:
# seasons = np.arange(2010,2023,1).astype(str)
# dfa = []
# for season in seasons:
#     df = get_ratings(season)
#     dfa.append(df)
# data = pd.concat(dfa)

In [ ]:
data = get_ratings(2023,ngames=10)

In [ ]:
train_x, train_y = convert_to_matricies(data, "ORtg1", teams_list, scale=1 / 2)
n = 1.5
n = 1.5/2
lambdas_net = [0.01 * n, 0.05 * n, 0.1 * n]
# lambdas_net = [10/3]
results_adj, model, intercept = calculate_netrtg(
    train_x, train_y, lambdas_net, teams_list
)
results, ortg_mean, drtg_mean = process_results(data, results_adj)
results.index = results.index +1
# results.to_csv("./NBA_Adj_Ratings.csv",index=False)
print(intercept)
# results.head(5)
# results.sort_values(by="aOFF",ascending=0)
# results.sort_values(by="aDEF",ascending=1)
results.sort_values("aNET",ascending=False)

In [ ]:
# fghfgh

In [ ]:
var1 = "NET"
# var1 = "OFF"
# var1 = "DEF"
var2 = "a" + var1
slope, intercept, r, p, sterr = scipy.stats.linregress(x=results[var1], y=results[var2])
r2 = r**2
fig, ax = plt.subplots(1, 1)
fig = sns.regplot(
    x=var1,
    y=var2,
    data=results,
    color="black",
    scatter_kws={"color": "tab:blue"},
    ax=ax,
)
ax.text(0.05, 0.9, r"$r^2=$" + f"{round(r2,4)}", transform=ax.transAxes)
ax.set_title("Adjusted Net Ratings vs Unadjusted")
plt.savefig("../figs/team_leaders/aNET_R2_1.png")
plt.show()

In [ ]:
results["OFF_R"] = results["OFF"].rank(ascending=False  ,method="first").astype(int)
results["DEF_R"] = results["DEF"].rank(ascending=True   ,method="first").astype(int)
results["NET_R"] = results["NET"].rank(ascending=False  ,method="first").astype(int)
results["aOFF_R"] = results["aOFF"].rank(ascending=False,method="first").astype(int)
results["aDEF_R"] = results["aDEF"].rank(ascending=True ,method="first").astype(int)
results["aNET_R"] = results["aNET"].rank(ascending=False,method="first").astype(int)

In [ ]:
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")
df_teams = df_teams.rename(columns={"nameTeam": "Team"})
results_plot = pd.merge(results, df_teams)

In [ ]:
black1 = '#252525'
black2 = '#222222'

In [ ]:
results_plot

In [ ]:
p = (ggplot(results_plot)
 # Slight modifications for the original lines,
 # 1. Nudge the text to either sides of the points
 # 2. Alter the color and alpha values
 + geom_text(aes(1, 'NET_R', label='Team' ), nudge_x=-0.05, ha='right', size=10, color=black1)
 + geom_text(aes(2, 'aNET_R', label='Team'), nudge_x=0.05, ha='left', size=10, color=black1)
 + geom_point(aes(1, 'NET_R', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_point(aes(2, 'aNET_R', color='colorsTeam'), size=2.5, alpha=.7)
 + geom_segment(aes(x=1, y='NET_R', xend=2, yend='aNET_R', color='colorsTeam'), alpha=.7, size=2)

 # Text Annotations
 #+ annotate('text', x=1, y=0, label='Rank in 1960', fontweight='bold', ha='right', size=10, color=black2)
 #+ annotate('text', x=2, y=0, label='Rank in 2015', fontweight='bold', ha='left', size=10, color=black2)
#  + annotate('text', x=1.5, y=0, label='Lines show change in rank', size=11, color=black1)
 + annotate('text', x=0.8, y=0, label='Unadjusted Net Rating Rank', size=11, color=black1)
 + annotate('text', x=2.2, y=0, label='SoS Adjusted Net Rating Rank', size=11, color=black1)
 #+ annotate('label', x=1.5, y=3, label='Lower infant\ndeath rates', size=9, color=black1,
 #           label_size=0, fontstyle='italic')
 #+ annotate('label', x=1.5, y=33, label='Higher infant\ndeath rates', size=9, color=black1,
 #           label_size=0, fontstyle='italic')

 # Prevent country names from being chopped off
 + lims(x=(0.35, 2.65))
#  + labs(color='Income Group')
 # Countries with lower rates on top
 + scale_y_reverse(breaks=np.arange(1,31,1))
 # Change colors
 + scale_color_identity(aesthetics=["color"])
 # Removes all decorations
 + theme_idv
 # Changing the figure size prevents the country names from squishing up
 + theme(figure_size=(8, 11))
 + theme(
     axis_title_x = element_blank(),
     axis_text_x = element_blank(),
     plot_title=element_text(face="bold", size=20),
 )
 + labs(
        title=f"Unadjusted vs SoS Adjusted Net Ratings",
        subtitle = "Last 10 games only",
        caption="@SravanNBA | source:nba.com/stats",
        y=f"Net Rating",
 )
)
p.save("../figs/team_ratings/Teams_aNET_mvmt.png", dpi=300, verbose = False)
p

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i results_plot
results <- results_plot
library(tidyverse)
library(ggimage)
library(ggrepel)
theme_owen <- function() {
  theme_minimal(base_size = 16, base_family = "Consolas") %+replace%
    theme(
      panel.grid.minor = element_blank(),
      plot.background = element_rect(fill = "ghostwhite", color = "ghostwhite")
    )
}
p <- ggplot(
  results,
  aes(x = aOFF, y = aDEF, label = paste0("#", aNET_R, " Net"))
) +
  # geom_point(aes(size = aNRtg_Rank)) +
  scale_y_reverse() +
  geom_hline(aes(yintercept = mean(aOFF)), color = "black") +
  geom_vline(aes(xintercept = mean(aDEF)), color = "black") +
  # geom_abline(intercept = 222, slope = -1, color = "black") +
  geom_abline(slope = -1,color="black")+
  geom_image(
    aes(
      x = aOFF, y = aDEF,
      image = urlThumbnailTeam
    ),
    size = 0.1
  ) +
  # geom_text(nudge_x = 1.3, nudge_y = 0, size = 6,check_overlap = TRUE) +
  geom_text_repel(nudge_x = 1, nudge_y = 0.5,size=6,min.segment.length=10) +
  # geom_label(nudge_x = 1.3, nudge_y = 0, size = 6) +
  theme_owen() +
  theme(
    plot.title.position = "plot",
    plot.title = element_text(face = "bold", size = 24, hjust = 0.5),
    plot.margin = margin(10, 10, 15, 10),
    plot.subtitle = element_text(size = 18),
    plot.caption = element_text(size = 14)
  ) +
  theme(
    axis.text.x = element_text(size = 14, face = "bold", color = "black"),
    axis.text.y = element_text(size = 14, face = "bold", color = "black"),
    axis.title.x = element_text(size = 18, face = "bold", colour = "black"),
    axis.title.y = element_text(size = 18, face = "bold", colour = "black")
  ) +
  labs(
    title = paste0("Adjusted Efficiency Landscape as of ", format(Sys.Date(), format = "%B %d, %Y")),
    x = "Adjusted Offensive Rating", y = "Adjusted Defensive Rating",
    subtitle = "Net Ratings here are adjusted for Strength of Schedule | Last 10 Games Only",
    caption = "@SravanNBA"
  )
ggsave("../figs/team_ratings/Adjusted_TRatings.png", p, w = 10 * 1.5, h = 8 * 1.5, dpi = 300)

In [ ]:
r1 = results[
    ["Team", "aOFF_R", "aDEF_R", "aNET_R", "OFF_R", "DEF_R", "NET_R"]
].reset_index(drop=True)

In [ ]:
r1["aOFF_m"] = r1["OFF_R"] - r1["aOFF_R"]
r1["aDEF_m"] = r1["DEF_R"] - r1["aDEF_R"]
r1["aNET_m"] = r1["NET_R"] - r1["aNET_R"]

In [ ]:
r2 = r1[
    ["Team", "aOFF_R", "aOFF_m", "aDEF_R", "aDEF_m", "aNET_R", "aNET_m"]
].reset_index(drop=True)

In [ ]:
%%R -i r2
library(tidyverse)
library(gt)
df <- r2
df %>% 
  gt()%>%
  tab_header(
    title = md("**NBA Adjusted Rating Movement 2023-24**"),
    subtitle = "R: Rank, M:Movement" 
    ) %>%
    data_color(columns = c(aOFF_m,aDEF_m,aNET_m), palette = c("red", "green")) %>%
    cols_align(align = "center",columns = c(aOFF_R,aOFF_m,aDEF_R,aDEF_m,aNET_R,aNET_m))  %>%
    cols_label(
      aOFF_R = "R", aDEF_R = "R", aNET_R = "R",
      aOFF_m = "M", aDEF_m = "M", aNET_m = "M"
    ) %>%
    tab_spanner(
      label = "OFF",
      columns = c(aOFF_R, aOFF_m)
    ) %>%
    tab_spanner(
      label = "DEF",
      columns = c(aDEF_R, aDEF_m)
    ) %>%
    tab_spanner(
      label = "NET",
      columns = c(aNET_R, aNET_m)
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_source_note(
    source_note = "Movement is calculated as unadjusted rank - adjusted rank" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/Teams_aNET_movement.png",zoom=5) 

In [ ]:
%%R -i results 
library(tidyverse)
library(gt)
df <- results
df %>% 
  gt()%>%
  # cols_move(OFF_R,OFF) %>% cols_move(DEF_R,DEF) %>% cols_move(NET_R,NET) %>%
  # cols_move(aOFF_R,aOFF) %>% cols_move(aDEF_R,aDEF) %>% cols_move(aNET_R,aNET) %>%
  tab_header(
    title = md("**NBA Adjusted Net Rating Leaders 2023-24**"),
    subtitle = md("The Offensive, Defensive and Net Ratings are adjusted for Strength of Schedule" )
    ) %>%
    data_color(columns = c(OFF,aOFF,NET,aNET), palette = c("red", "green")) %>%
    data_color(columns = c(DEF,aDEF), palette = c("green","red")) %>%
    cols_align(align = "center",columns = c(OFF,DEF,NET,aOFF,aDEF,aNET))  %>%
    cols_label(
      OFF_R = "#",DEF_R = "#",NET_R = "#",aOFF_R = "#",aDEF_R = "#",aNET_R = "#"
    ) %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 12,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 10,
        heading.title.font.size = 20,
        heading.subtitle.font.size = 12.5,
        table.font.names = "Consolas", 
        data_row.padding = px(2)
    ) %>% 
    tab_spanner(
      label = "Unadjusted",
      columns = c(OFF, OFF_R, DEF, DEF_R, NET, NET_R)
    ) %>%
    tab_spanner(
      label = "Adjusted",
      columns = c(aOFF, aOFF_R, aDEF, aDEF_R, aNET, aNET_R)
    ) %>%
    tab_source_note(
    source_note = "SOS adjustments done using Ridge Regression Method to estimate Off and Def Ratings" ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/Teams_aNET.png",zoom=5) 
